# Assignment 4: Pipelines and Text Data (60 total marks)
### Due: March 21 at 11:59pm

### Name: 

In [144]:
import numpy as np
import pandas as pd

In [145]:
import warnings
warnings.filterwarnings('ignore') #ignoring some deprication warnings

## Part 1: Pipelines (26 marks)

The purpose of this part of the assignment is to practice following the grid-search workflow: 
- Split data into training and test set
- Use the training portion to find the best model using grid search and cross-validation
- Retrain the best model
- Evaluate the retrained model on the test set

### 1.1: Load data (4 marks)
For this task, we will be using stock data from the Dow Jones Index. This dataset uses information about different stocks to try to predict what the percent change in price will be from week to week.

More information on the dataset can be found here: https://archive.ics.uci.edu/dataset/312/dow+jones+index

In [146]:
# TO DO: Load the dataset into a dataframe called stock_data (0.5 marks)
stock_data = pd.read_csv('dow_jones_index.data')

# TO DO: Inspect the first few columns (0.5 marks)

print(stock_data.head())


   quarter stock       date    open    high     low   close     volume  \
0        1    AA   1/7/2011  $15.82  $16.72  $15.78  $16.42  239655616   
1        1    AA  1/14/2011  $16.71  $16.71  $15.64  $15.97  242963398   
2        1    AA  1/21/2011  $16.19  $16.38  $15.60  $15.79  138428495   
3        1    AA  1/28/2011  $15.87  $16.63  $15.82  $16.13  151379173   
4        1    AA   2/4/2011  $16.18  $17.39  $16.18  $17.14  154387761   

   percent_change_price  percent_change_volume_over_last_wk  \
0               3.79267                                 NaN   
1              -4.42849                            1.380223   
2              -2.47066                          -43.024959   
3               1.63831                            9.355500   
4               5.93325                            1.987452   

   previous_weeks_volume next_weeks_open next_weeks_close  \
0                    NaN          $16.71           $15.97   
1            239655616.0          $16.19           $15

In [147]:
# TO DO: Check the data types of each column and if there are missing values (0.5 marks)

print(stock_data.dtypes)
print(stock_data.isnull().sum())


quarter                                 int64
stock                                  object
date                                   object
open                                   object
high                                   object
low                                    object
close                                  object
volume                                  int64
percent_change_price                  float64
percent_change_volume_over_last_wk    float64
previous_weeks_volume                 float64
next_weeks_open                        object
next_weeks_close                       object
percent_change_next_weeks_price       float64
days_to_next_dividend                   int64
percent_return_next_dividend          float64
dtype: object
quarter                                0
stock                                  0
date                                   0
open                                   0
high                                   0
low                                    0
clos

You should notice in this dataset that there are multiple columns that look numerical, but include a `$` that turns the value into a string (type object). You can use the code below to convert these columns into numerical ones:

In [148]:
# TO DO: Fill-in which columns need the $ to be removed (1 mark)
columns = ['open', 'high', 'low', 'close', 'next_weeks_open', 'next_weeks_close']   

# Code to remove $ - DO NOT CHANGE
stock_data[columns] = stock_data[columns].replace('[\$]', '', regex=True).astype(float)

# TO DO: Inspect first few rows to make sure it worked (0.5 marks)
print(stock_data.head())


   quarter stock       date   open   high    low  close     volume  \
0        1    AA   1/7/2011  15.82  16.72  15.78  16.42  239655616   
1        1    AA  1/14/2011  16.71  16.71  15.64  15.97  242963398   
2        1    AA  1/21/2011  16.19  16.38  15.60  15.79  138428495   
3        1    AA  1/28/2011  15.87  16.63  15.82  16.13  151379173   
4        1    AA   2/4/2011  16.18  17.39  16.18  17.14  154387761   

   percent_change_price  percent_change_volume_over_last_wk  \
0               3.79267                                 NaN   
1              -4.42849                            1.380223   
2              -2.47066                          -43.024959   
3               1.63831                            9.355500   
4               5.93325                            1.987452   

   previous_weeks_volume  next_weeks_open  next_weeks_close  \
0                    NaN            16.71             15.97   
1            239655616.0            16.19             15.79   
2          

In [149]:
# TO DO: Check data type of each column to make sure that the type of the columns selected has changed (0.5 marks)
print(stock_data.dtypes)


quarter                                 int64
stock                                  object
date                                   object
open                                  float64
high                                  float64
low                                   float64
close                                 float64
volume                                  int64
percent_change_price                  float64
percent_change_volume_over_last_wk    float64
previous_weeks_volume                 float64
next_weeks_open                       float64
next_weeks_close                      float64
percent_change_next_weeks_price       float64
days_to_next_dividend                   int64
percent_return_next_dividend          float64
dtype: object


The first thing we need to do is deal with missing values. Looking at the dataset, there are two columns with 30 missing values. For this case, we will drop these rows instead of filling them in.

In [150]:
# TO DO: Drop rows with missing data (0.5 marks)

stock_data = stock_data.dropna()

print(stock_data.head())


   quarter stock       date   open   high    low  close     volume  \
1        1    AA  1/14/2011  16.71  16.71  15.64  15.97  242963398   
2        1    AA  1/21/2011  16.19  16.38  15.60  15.79  138428495   
3        1    AA  1/28/2011  15.87  16.63  15.82  16.13  151379173   
4        1    AA   2/4/2011  16.18  17.39  16.18  17.14  154387761   
5        1    AA  2/11/2011  17.33  17.48  16.97  17.37  114691279   

   percent_change_price  percent_change_volume_over_last_wk  \
1             -4.428490                            1.380223   
2             -2.470660                          -43.024959   
3              1.638310                            9.355500   
4              5.933250                            1.987452   
5              0.230814                          -25.712195   

   previous_weeks_volume  next_weeks_open  next_weeks_close  \
1            239655616.0            16.19             15.79   
2            242963398.0            15.87             16.13   
3          

### 1.2: Pre-processing (4 marks)

In this dataset, we have columns with:
- Categorical values
- Numerical values

We need to create a column transformer that will use the proper preprocessing methods on each type of column.

In [151]:
# TO DO: Create Column Transformer using an encoder and StandardScaler (1 mark)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = ColumnTransformer(
    [("scaling", StandardScaler(), ['quarter', 'open', 'high', 'low', 'close', 'volume', 'percent_change_price', 'next_weeks_open', 'next_weeks_close', 'percent_change_next_weeks_price', 'days_to_next_dividend', 'percent_return_next_dividend']),
     ("onehot", OneHotEncoder(sparse_output=False), ['stock', 'date'])])


In [152]:
# TO DO: Initialize your pipeline with your column transformer and the Ridge Regression model (1 mark)
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

pipe = Pipeline(steps=[('preprocessor', ct), ('classifier', Ridge())])

In [153]:
# TO DO: Separate data into feature matrix and target vector (1 mark)

X = stock_data.drop('percent_change_next_weeks_price', axis=1)  
y = stock_data['percent_change_next_weeks_price']


In [154]:
# TO DO: Split data into training and testing sets (use random_state=0 and 10% of the data for testing) (0.5 marks)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

Create another column transformer that does not implement scaling

In [155]:
# TO DO: Create a new column transformer that only performs encoding (0.5 marks)

ct_encoding = ColumnTransformer(
    [("onehot", OneHotEncoder(sparse_output=False), ['stock', 'date'])])


### 1.3: Grid Search (4 marks)

For the grid search, we want to compare the performance of the Random Forest model to a Ridge Regression model with the two different column transformers. Think about if we need to use scaling for both models. Select parameter values to test that make sense for both models.

In [156]:
# TO DO: Create parameter grid and initialize grid object (3 marks)

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier

param_grid = [{'classifier': [KNN()],
              'classifier__n_neighbors': [1, 3, 5, 7, 9]},
             {'classifier': [RandomForestClassifier()],
              'classifier__n_estimators': [10, 50, 100],
              'classifier__max_depth': [None, 10, 20]}]


In [157]:
# TO DO: Fit grid object to training data (1 mark)

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, param_grid, cv=5, return_train_score=True)

grid.fit(X_train, y_train)

ValueError: 
All the 70 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'percent_change_next_weeks_price'

Th

### 1.4: Visualize Results (2 marks)

The final step is to print out the results from the grid search. You will need to print out the following items:
- Best parameters
- Best cross-validation train score 
- Best cross-validation test score
- Test set accuracy

In [158]:
# TO DO: Print the results from the grid search (2 marks)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid.cv_results_['mean_train_score'][grid.best_index_]))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

### Questions (8 marks)

1. Which models did you use scaling for? Why?
1. Which model and what parameters produced the best results?
1. Was this model a good fit? Why or why not?
1. Is there anything else we could do to try to improve model performance? Provide two ideas.

*ANSWER HERE*


### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE - BE SPECIFIC*

## Part 2: Text Data (32 marks)

The purpose of this part of the assignment is to practice working with text data.

### 2.1: Load data (1 mark)
For this task, we will be using the hobbies dataset from the yellowbrick library. More information on the dataset can be found here: https://www.scikit-yb.org/en/latest/api/datasets/hobbies.html

In [163]:
%pip install setuptools
%pip install yellowbrick
# TO DO: Install yellowbrick (0.5 marks)
from yellowbrick.datasets import load_hobbies

# Load the hobbies dataset
data = load_hobbies()
print(data.head())


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'distutils'

### 2.2 Pre-processing (3 marks)

We will need to transform the data from strings to numeric. First, we will transform the data using `CountVectorizer(min_df=5)`.

In [130]:
# TO DO: Create CountVectorizer object (0.5 marks)

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=5)



In [128]:
# TO DO: Fit vectorizer to data (0.5 marks)

vect.fit()


KeyError: 'news_headlines'

In [ ]:
# TO DO: What is the length of the vocabulary? (0.5 marks)


In [ ]:
# TO DO: Transform the data (0.5 marks)


In [ ]:
# TO DO: What is the shape of the transformed data? (0.5 marks)


In [ ]:
# TO DO: Split data into training and testing sets (use random_state=0 and 10% of the data for testing) (0.5 marks)


### 2.3: Grid Search (5 marks)

For the grid search, we want to compare the performance of Logistic Regression for different values of C. Initialize the parameter grid with parameter values that make sense for this model.

In [ ]:
# TO DO: Create parameter grid and initialize grid object (2 marks)


In [ ]:
# TO DO: Fit grid object to training data (1 mark)


In [ ]:
# TO DO: Print the results from the grid search (2 marks)


### 2.4: Additional Model Comparisons (9 marks)

### 2.4.1: Naive Bayes (3 marks)
We would like to compare the performance of Logistic Regression with one of the Naive Bayes models. Pick the Naive Bayes model that you think would best suit text data and implement below. Since we are not adjusting hyperparameters, we can use `cross_validate`.

In [ ]:
# TO DO: Implement Naive Bayes model with cross-validate (2 marks)


# TO DO: Print training and validation accuracies


In [ ]:
# TO DO: Calculate and print test accuracy (1 mark)


### 2.4.2 Tf-idf (6 marks)

To try to improve the results, we can try using Tf-idf to tranform the text data based on the importance of each feature. We will need to use a pipeline and the original data for this section. Use `TfidfVectorizer(min_df=5)` and compare the results for both Logistic Regression and your selected Naive Bayes model. Use the Logistic Regression parameters from the previous section.

In [ ]:
# TO DO: Split the data into training and testing sets (same values as previous section) (1 mark)


In [ ]:
# TO DO: Implement Pipeline with Tf-idf vectorizer and both Logistic Regression and your selected Naive Bayes model (3 marks)


In [ ]:
# TO DO: Print the results from the grid search (2 marks)


### Questions (10 marks)

1. Which Naive Bayes model did you pick? Why?
1. Which model and what parameters produced the best results?
1. Was this model a good fit? Why or why not?
1. Is there anything else we could do to try to improve model performance? Provide two ideas (must be different from Part 1).
1. Why did we need to implement a pipeline for Tf-idf and not CountVectorizer? What would happen if we didn't use one for Tf-idf?

*ANSWER HERE*


### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE - BE SPECIFIC*

## Part 3: Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challangeing, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*